In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [3]:
# itp_id = 4 ## AC Transit
itp_id = 159 ## Lake Transit

In [4]:
import numpy as np

In [7]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [8]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

# Generate analysis data for all dates

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
rt_analysis = rt.OperatorDayAnalysis(itp_id, dt.date(2022, 6, 1), pbar)
rt_analysis.export_views_gcs()

found parquet
found parquet
found parquet
found parquet
found parquet


In [16]:
shared_utils.rt_utils.get_vehicle_positions(159, dt.date(2022, 5, 7))

AssertionError: no vehicle positions data found for 2022-05-07

In [12]:
shared_utils.rt_utils.get_trips(159, dt.date(2022, 4, 30))

found parquet


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_long_name,route_short_name,route_desc,route_type
0,159,0,2022-04-30,9099194876581177237,t_1547389_b_25985_tn_0,19311,0,p_1275849,2021-10-04,2022-09-06,Lakeport City,8,None,3
1,159,0,2022-04-30,-3782819459302039695,t_1439288_b_25985_tn_0,19311,0,p_1275849,2021-06-21,2099-01-01,Lakeport City,8,None,3
2,159,0,2022-04-30,-8925727387365668020,t_1439285_b_25985_tn_0,19311,0,p_1275849,2021-06-21,2022-09-06,Lakeport City,8,None,3
3,159,0,2022-04-30,1149773370943146805,t_1439296_b_25985_tn_0,19311,0,p_1275848,2021-06-21,2022-09-06,Lakeport City,8,None,3
4,159,0,2022-04-30,3807771985519687851,t_1439297_b_25985_tn_0,19311,0,p_1275848,2021-06-21,2099-01-01,Lakeport City,8,None,3
5,159,0,2022-04-30,-1724005930502949966,t_1439305_b_25985_tn_0,19311,0,p_1275848,2021-06-21,2022-09-06,Lakeport City,8,None,3
6,159,0,2022-04-30,-7278269088169232829,t_1439300_b_25985_tn_0,19311,0,p_1275848,2021-06-21,2099-01-01,Lakeport City,8,None,3
7,159,0,2022-04-30,6651208253536497037,t_1439291_b_25985_tn_0,19311,0,p_1275849,2021-06-21,2099-01-01,Lakeport City,8,None,3
8,159,0,2022-04-30,8652216044727557938,t_1439292_b_25985_tn_0,19311,0,p_1275849,2021-06-21,2099-01-01,Lakeport City,8,None,3
9,159,0,2022-04-30,5854489612375951727,t_1439302_b_25985_tn_0,19311,0,p_1275848,2021-06-21,2099-01-01,Lakeport City,8,None,3


In [8]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
        rt_analysis.export_views_gcs()
        print(f'complete for date: {date}')
    except Exception as e:
        print(e)
        print(f'failed for date: {date}')
        continue

found parquet
found parquet
found parquet
found parquet
vehicle positions trip ids not in schedule
failed for date: 2022-04-30
found parquet
found parquet
cached parquet empty, will try a fresh query
getting trips...
found parquet
cached parquet empty, will try a fresh query
found parquet
vehicle positions trip ids not in schedule
failed for date: 2022-05-01
found parquet
found parquet
found parquet
found parquet
found parquet
complete for date: 2022-05-02
found parquet
found parquet
found parquet
found parquet
found parquet
could not generate delays for trip t_1057273_b_25985_tn_0
Columns must be same length as key
could not generate delays for trip t_1547367_b_25985_tn_0
Columns must be same length as key
complete for date: 2022-05-03
found parquet
found parquet
found parquet
found parquet
found parquet
could not generate delays for trip t_1439208_b_25985_tn_0
'float' object has no attribute 'date'
could not generate delays for trip t_1547206_b_25985_tn_0
Columns must be same length 

In [17]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

failed for date: 2022-04-30
failed for date: 2022-05-01
found parquet
complete for date: 2022-05-02
found parquet
complete for date: 2022-05-03
found parquet
complete for date: 2022-05-04
found parquet
complete for date: 2022-05-05
found parquet
complete for date: 2022-05-06
failed for date: 2022-05-07
failed for date: 2022-05-08


In [35]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 4))

found parquet


In [9]:
# corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson') ## alameda
corridor = gpd.read_file('./lakeport_map.geojson') ## lakeport

In [37]:
# corridor.explore()

In [38]:
rt_day.add_corridor(corridor)

In [39]:
to_map = rt_day.corridor_stop_delays >> distinct(_.stop_name, _.geometry)

In [40]:
corridor

,geometry
0,"POLYGON ((-122.78458 38.89638, -122.61223 38.8..."


In [46]:
to_map = to_map.to_crs(shared_utils.geography_utils.WGS84).dropna()

In [47]:
to_map

,stop_name,geometry
0,Kit's Corner,POINT (-122.74312 38.93055)
2,Main & Fourth,POINT (-122.83661 38.97778)
3,Hwy. 53 and Hwy. 29,POINT (-122.61243 38.91147)
4,Burns Valley Mall,POINT (-122.63390 38.96176)
5,"Foods, Etc./Clearlake Family Clinic",POINT (-122.62854 38.94996)
6,Phillips Avenue and 40th Avenue,POINT (-122.61510 38.95091)
7,Lakeshore Drive and Austin Road (Austin Park),POINT (-122.64923 38.95976)
8,Phillips Avenue and 40th Avenue,POINT (-122.61530 38.94981)
9,Senior Center,POINT (-122.63113 38.96600)
10,Veteran's Clinic Lakeshore,POINT (-122.62870 38.94979)


In [49]:
pd.concat([to_map, corridor]).explore(tiles = "CartoDB positron")

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops
* Assert that more than 50% of filtered stops within corridor, if not then drop shape (remove shapes starting/ending in corridor but mostly running outside)

# Report:

* Sum of median trip delays
* Median segment speed
* Delay compared to speed standards: 16mph (approx. 75th %ile for medium size operator by 2010 NTD)

# Speed-based Metrics

In [10]:
pbar = tqdm()

0it [00:00, ?it/s]

In [11]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [ ]:
m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

In [ ]:
m

In [50]:
may4_metrics = rt_filter_map_plot.RtFilterMapper.corridor_metrics(rt_day)

In [51]:
may4_metrics

{'schedule_metric': 6.075, 'speed_metric': -226.84421784548465}

In [56]:
##TODO something hacky to drop the first part of route 4, in pre-processing... (ie in vp trips...)

In [12]:
# rt_day.segment_speed_map(corridor=True)

In [13]:
# rt_day.rt_trips >> filter(_.route_short_name == '4')

# Corridor-based metrics for SCCP

* intent is to eventually parameterize this and output formatted report, also methodology may change

In [14]:
pbar = tqdm()

0it [00:00, ?it/s]

In [15]:
schedule_metrics = []
speed_metrics = []
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
        rt_day.add_corridor(corridor)
        _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
        metrics = rt_day.corridor_metrics()
        schedule_metrics += [metrics['schedule_metric']]
        speed_metrics += [metrics['speed_metric']]
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

failed for date: 2022-04-30
failed for date: 2022-05-01
found parquet
complete for date: 2022-05-02
found parquet
complete for date: 2022-05-03
found parquet
complete for date: 2022-05-04
found parquet
complete for date: 2022-05-05
found parquet
complete for date: 2022-05-06
failed for date: 2022-05-07
failed for date: 2022-05-08


In [16]:
np.round(np.mean(schedule_metrics), 0)

8.0

In [17]:
np.round(np.mean(speed_metrics), 0)

-283.0